In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train', names=['item'])
test_data = pd.read_csv('test', names=['item'])

In [3]:
df = pd.concat([train_data, test_data], ignore_index=True)


<h2>Преобразуем датафрейм к виду {user, item, order}</h2>

In [ ]:
df['order'] = df['item'].apply(lambda x: list(range(len(x.split(' '))-1, -1, -1)))

In [5]:
df = df.assign(liked_id=df['liked_id'].str.split(' '))

    

In [6]:
df = df.explode(['liked_id', 'order'])
df['index_col'] = df.index
df = df.reset_index()
df = df.drop('index_col', axis=1)
df = df.rename(columns={"index": "user"})

## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [ ]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, user_column, item_column, random_seed=23):
        self.n_folds = n_folds
        self.random_seed = random_seed
        self.user_column = user_column
        self.item_column = item_column
        self.p = p
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            grouped_df = (df.groupby([self.user_column]).count() <= self.p).reset_index()
            p_test_users = grouped_df.loc[grouped_df[self.item_column]==True][self.user_column].to_list()
            test_mask = df[self.user_column].isin(set(test_fold_users) & set(p_test_users))
            train_mask = ~test_mask
            
            yield train_mask, test_mask

In [69]:
cv = UsersKFoldPOut(n_folds=3, p=1, user_column='user', item_column='item')


In [71]:
next(iter(cv.split(df)))

(0        True
 1        True
 2        True
 3        True
 4        True
          ... 
 38110    True
 38111    True
 38112    True
 38113    True
 38114    True
 Name: user, Length: 38115, dtype: bool,
 0        False
 1        False
 2        False
 3        False
 4        False
          ...  
 38110    False
 38111    False
 38112    False
 38113    False
 38114    False
 Name: user, Length: 38115, dtype: bool)